In [75]:
from tqdm.notebook import tqdm, trange

In [76]:
tqdm.pandas()

In [3]:
import sys

In [4]:
import glob
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [5]:
import os

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import PIL.Image

from IPython.display import Image, display
import warnings
warnings.filterwarnings("ignore")

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/barzamini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/barzamini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def find_gpus(nums=6):
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp_free_gpus')
    with open('tmp_free_gpus', 'r') as lines_txt:
        frees = lines_txt.readlines()
        idx_freeMemory_pair = [ (idx,int(x.split()[2]))
                              for idx,x in enumerate(frees) ]
    idx_freeMemory_pair.sort(key=lambda my_tuple:my_tuple[1],reverse=True)
    usingGPUs = [str(idx_memory_pair[0])
                    for idx_memory_pair in idx_freeMemory_pair[:nums] ]
    usingGPUs =  ','.join(usingGPUs)
    print('using GPU idx: #', usingGPUs)
    return usingGPUs

In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = find_gpus(nums=2)

using GPU idx: # 0,1


In [9]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## get topics

In [36]:
from bertopic import BERTopic

In [37]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [38]:
lemmatizer = WordNetLemmatizer()

In [41]:
def get_topics(concept='pedestrian'):
    if concept == 'pedestrian':
        model_path = 'ped_auto_model'
    elif concept == 'aircraft':
        model_path = 'aircraft_auto_model'
    elif concept == 'car':
        model_path = 'car_auto_model'
    auto_model = BERTopic(nr_topics="auto")
    auto_model = BERTopic.load(model_path)
    similar_topics, similarity = auto_model.find_topics(concept, top_n=50)

    topics = dict()
    for t in similar_topics:
        l1 = auto_model.get_topic(t)
        el_list = []
        el_list.append(concept)
        for el in l1:
            w = lemmatizer.lemmatize(el[0], get_wordnet_pos(el[0]))
            el_list.append(w)
        topics[t]= set(el_list)
    return topics

# Load files

In [11]:
print(os.listdir('/raid/AISSEL/Hamed/datasets/wit/'))

['wit_v1.train.all-00007-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_c.tsv', '__MACOSX', 'wit_v1.train.all-1percent_sample.tsv', 'wit_v1.train.all-00000-of-00010_en_csim.tsv', 'images', 'selected_url.tsv', 'selected_url_1.tsv', 'selected_url_2.tsv', 'selected_url_3.tsv', 'selected_url_4.tsv', 'selected_url_5.tsv', 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00001-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00002-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00003-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00004-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00005-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00006-of-00010_context_caption_en_sbert_c.tsv', 'wit_v1.train.all-00008-of-00010_context_caption_en_sbert_c.tsv']


In [12]:
root_path = '/raid/AISSEL/Hamed/datasets/wit'

In [ ]:
img_dirs = ['wit_305']
dst = root_path + '/' + img_dest
for i_dir in img_dirs:
#     print(i_dir)
    src = root_path + '/' + i_dir
    l1 = [name for name in os.listdir(src) if os.path.isfile(os.path.join(src, name))]
#     print(len(l1))
    for fn in tqdm(l1, total=len(l1)):
        try:
            shutil.move(src + '/' + fn , dst)
        except Exception as e:
            print(i_dir)
            print(e)
            pass
#         break
#     break


In [13]:
f_names =[el for el in os.listdir('/raid/AISSEL/Hamed/datasets/wit/') if el.endswith('_en_sbert_c.tsv')]
f_names

['wit_v1.train.all-00007-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00009-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00000-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00001-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00002-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00003-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00004-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00005-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00006-of-00010_context_caption_en_sbert_c.tsv',
 'wit_v1.train.all-00008-of-00010_context_caption_en_sbert_c.tsv']

In [14]:
df = pd.read_csv(f'{root_path}/{f_names[0]}', sep='\t')
df = df.drop('Unnamed: 0', 1)
df = df[df["language"]=='en']
df

,index,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,...,192_context_score,20_context_score,296_context_score,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,concept_caption_score,concept_context_score
0,8,en,https://en.wikipedia.org/wiki/Bianca_Atzei,https://upload.wikimedia.org/wikipedia/commons...,Bianca Atzei,Sanremo Music Festival 2015 and Bianco e nero,Bianca Atzei / Career / Sanremo Music Festival...,Bianca Atzei's signature,Italiano: Firma di Bianca Atzei,NaN,...,-0.038851,0.089934,-0.015047,0.043425,0.027713,-0.098772,0.032884,-0.122536,0.090904,0.040136
1,16,en,https://en.wikipedia.org/wiki/International_Co...,https://upload.wikimedia.org/wikipedia/commons...,International Committee of the Red Cross archives,World War II,International Committee of the Red Cross archi...,The Agency archives at Plainpalais,Nederlands: Collectie / Archief&#160;: Fotocol...,NaN,...,-0.091849,0.002981,0.042102,0.053072,-0.039682,-0.103131,0.016510,-0.044889,-0.008080,-0.010542
2,20,en,https://en.wikipedia.org/wiki/Flight_helmet,https://upload.wikimedia.org/wikipedia/commons...,Flight helmet,History of flight helmets,Flight helmet / History of flight helmets,NaN,English: A-9 oxygen mask with B-6 winter helme...,NaN,...,0.016546,-0.028779,-0.004041,0.086849,-0.038401,0.074521,0.059790,0.012366,0.118914,0.065914
3,22,en,https://en.wikipedia.org/wiki/List_of_burials_...,https://upload.wikimedia.org/wikipedia/commons...,List of burials at Serafimovskoe Cemetery,Sport,List of burials at Serafimovskoe Cemetery / In...,NaN,English: Russian coach Vladimir Kazachyonok Ру...,NaN,...,-0.081875,-0.031366,0.061323,-0.060698,-0.051773,0.031197,0.036632,-0.057523,0.027825,0.002939
4,29,en,https://en.wikipedia.org/wiki/List_of_Historic...,https://upload.wikimedia.org/wikipedia/commons...,List of Historic Sites of Japan (Okayama),Prefectural Historic Sites,List of Historic Sites of Japan (Okayama) / Pr...,NaN,"English: Top of kouen part and Stone coffin, T...",NaN,...,-0.066597,-0.022394,0.130063,-0.007258,-0.076814,0.081103,-0.060480,0.085220,-0.033427,-0.017023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541723,3703714,en,https://simple.wikipedia.org/wiki/Vibraye,http://upload.wikimedia.org/wikipedia/commons/...,Vibraye,NaN,Vibraye,coat of arms,NaN,NaN,...,-0.011430,0.004210,0.059705,-0.041704,0.002616,0.007550,-0.172665,-0.067527,0.230597,-0.065141
541724,3703716,en,https://en.wikipedia.org/wiki/Wilhelm_Schmiede...,https://upload.wikimedia.org/wikipedia/commons...,Wilhelm Schmiedeberg,NaN,Wilhelm Schmiedeberg,Wilhelm Schmiedeberg \n Self portrait of a mir...,Deutsch: seitenverkehrtes Selbstbildnis von Wi...,NaN,...,0.036386,0.020026,-0.041863,0.040273,-0.018524,-0.067019,0.127569,0.030676,0.163752,0.059812
541725,3703719,en,https://en.wikipedia.org/wiki/Political_system...,https://upload.wikimedia.org/wikipedia/commons...,Political systems of Imperial China,Three Lords and Nine Ministers system,Political systems of Imperial China / Central ...,NaN,A pottery model of a palace from the Chinese H...,NaN,...,0.065037,0.004745,-0.082471,-0.012358,-0.086395,0.022315,-0.032922,0.011763,0.008349,-0.030208
541726,3703724,en,https://en.wikipedia.org/wiki/Rehau,https://upload.wikimedia.org/wikipedia/commons...,Rehau,NaN,Rehau,Rehau,Deutsch: Ansicht der Stadt Rehau vom Kornberg,Rehau,...,-0.024338,-0.063756,0.006761,-0.049635,-0.086546,-0.120204,0.054417,-0.033827,0.192601,-0.026689


In [17]:
for el in df.columns:
    print(el)

index
language
page_url
image_url
page_title
section_title
hierarchical_section_title
caption_reference_description
caption_attribution_description
caption_alt_text_description
mime_type
original_height
original_width
is_main_image
attribution_passes_lang_id
page_changed_recently
context_page_description
context_section_description
caption
context
196_caption_score
412_caption_score
172_caption_score
363_caption_score
198_caption_score
114_caption_score
419_caption_score
294_caption_score
388_caption_score
314_caption_score
98_caption_score
80_caption_score
134_caption_score
432_caption_score
394_caption_score
53_caption_score
56_caption_score
150_caption_score
278_caption_score
-1_caption_score
43_caption_score
265_caption_score
345_caption_score
378_caption_score
11_caption_score
331_caption_score
202_caption_score
169_caption_score
327_caption_score
309_caption_score
68_caption_score
216_caption_score
40_caption_score
99_caption_score
426_caption_score
344_caption_score
47_caption_s

In [18]:
caption_score_list = [el for el in df.columns if 'caption_score' in el]
print(caption_score_list)

['196_caption_score', '412_caption_score', '172_caption_score', '363_caption_score', '198_caption_score', '114_caption_score', '419_caption_score', '294_caption_score', '388_caption_score', '314_caption_score', '98_caption_score', '80_caption_score', '134_caption_score', '432_caption_score', '394_caption_score', '53_caption_score', '56_caption_score', '150_caption_score', '278_caption_score', '-1_caption_score', '43_caption_score', '265_caption_score', '345_caption_score', '378_caption_score', '11_caption_score', '331_caption_score', '202_caption_score', '169_caption_score', '327_caption_score', '309_caption_score', '68_caption_score', '216_caption_score', '40_caption_score', '99_caption_score', '426_caption_score', '344_caption_score', '47_caption_score', '8_caption_score', '416_caption_score', '112_caption_score', '137_caption_score', '441_caption_score', '192_caption_score', '20_caption_score', '296_caption_score', '144_caption_score', '12_caption_score', '224_caption_score', '250_c

In [19]:
caption_score_list = ['196_caption_score', '412_caption_score', '172_caption_score', '363_caption_score', 
                      '198_caption_score', '114_caption_score', '419_caption_score', '294_caption_score', 
                      '388_caption_score', '314_caption_score', '98_caption_score', '80_caption_score', 
                      '134_caption_score', '432_caption_score', '394_caption_score', '53_caption_score', 
                      '56_caption_score', '150_caption_score', '278_caption_score', '-1_caption_score', 
                      '43_caption_score', '265_caption_score', '345_caption_score', '378_caption_score', 
                      '11_caption_score', '331_caption_score', '202_caption_score', '169_caption_score', 
                      '327_caption_score', '309_caption_score', '68_caption_score', '216_caption_score', 
                      '40_caption_score', '99_caption_score', '426_caption_score', '344_caption_score', 
                      '47_caption_score', '8_caption_score', '416_caption_score', '112_caption_score', 
                      '137_caption_score', '441_caption_score', '192_caption_score', '20_caption_score', 
                      '296_caption_score', '144_caption_score', '12_caption_score', '224_caption_score', 
                      '250_caption_score', '376_caption_score']

In [20]:
context_score_list = [el for el in df.columns if 'context_score' in el]
context_score_list = context_score_list[:-1]
print(context_score_list)

['196_context_score', '412_context_score', '172_context_score', '363_context_score', '198_context_score', '114_context_score', '419_context_score', '294_context_score', '388_context_score', '314_context_score', '98_context_score', '80_context_score', '134_context_score', '432_context_score', '394_context_score', '53_context_score', '56_context_score', '150_context_score', '278_context_score', '-1_context_score', '43_context_score', '265_context_score', '345_context_score', '378_context_score', '11_context_score', '331_context_score', '202_context_score', '169_context_score', '327_context_score', '309_context_score', '68_context_score', '216_context_score', '40_context_score', '99_context_score', '426_context_score', '344_context_score', '47_context_score', '8_context_score', '416_context_score', '112_context_score', '137_context_score', '441_context_score', '192_context_score', '20_context_score', '296_context_score', '144_context_score', '12_context_score', '224_context_score', '250_c

In [21]:
im_c = ['image_url', 'caption', 'context']
cap_s = ['concept_caption_score']
con_s = ['concept_context_score']
col_list = caption_score_list + context_score_list + im_c + cap_s + con_s

In [22]:

col_list_1 = context_score_list + im_c + con_s

In [23]:
csim_df = pd.DataFrame(columns = col_list)
for idx, fn in tqdm(enumerate(f_names)):
#     if idx == 2:
#         break
    df = pd.read_csv(f'{root_path}/{fn}', sep='\t')
    df = df.drop('Unnamed: 0', 1)
    df = df[df["language"]=='en']
    df = df[col_list]
#     df_1 = df_1.sort_values(by=['196_score'], ascending=False)
    df = df.reset_index(drop=True)
    csim_df = pd.concat([csim_df, df], ignore_index=True)
    

0it [00:00, ?it/s]

In [24]:
# csim_df.sort_values(by=['196_score'], ascending=False)
csim_df

,196_caption_score,412_caption_score,172_caption_score,363_caption_score,198_caption_score,114_caption_score,419_caption_score,294_caption_score,388_caption_score,314_caption_score,...,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,image_url,caption,context,concept_caption_score,concept_context_score
0,0.014023,0.048338,0.072041,0.035916,-0.046189,0.007926,0.033050,0.015909,0.051816,0.022079,...,0.043425,0.027713,-0.098772,0.032884,-0.122536,https://upload.wikimedia.org/wikipedia/commons...,Bianca Atzei's signature Italiano: Firma di Bi...,Veronica Atzei – better known by her stage nam...,0.090904,0.040136
1,-0.087317,-0.021710,-0.051332,-0.061621,0.014076,0.049088,-0.088426,-0.019032,-0.007908,-0.003852,...,0.053072,-0.039682,-0.103131,0.016510,-0.044889,https://upload.wikimedia.org/wikipedia/commons...,The Agency archives at Plainpalais Nederlands:...,The archives of the International Committee of...,-0.008080,-0.010542
2,-0.023712,0.000274,0.003510,-0.040632,-0.034856,0.131114,0.052585,0.048979,-0.074227,0.089140,...,0.086849,-0.038401,0.074521,0.059790,0.012366,https://upload.wikimedia.org/wikipedia/commons...,English: A-9 oxygen mask with B-6 winter helm...,"A flight helmet, sometimes referred to as a ""b...",0.118914,0.065914
3,-0.059316,-0.073104,-0.019552,-0.094266,0.022857,0.071975,-0.046809,-0.035883,0.035720,-0.060381,...,-0.060698,-0.051773,0.031197,0.036632,-0.057523,https://upload.wikimedia.org/wikipedia/commons...,English: Russian coach Vladimir Kazachyonok Р...,Serafimovskoe Cemetery in the north-west of Sa...,0.027825,0.002939
4,-0.100309,-0.125802,-0.069639,-0.151786,-0.100653,-0.070315,-0.109904,-0.068974,-0.122775,-0.067142,...,-0.007258,-0.076814,0.081103,-0.060480,0.085220,https://upload.wikimedia.org/wikipedia/commons...,"English: Top of kouen part and Stone coffin, ...",This list is of the Historic Sites of Japan lo...,-0.033427,-0.017023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5411973,0.010525,0.086484,0.072844,0.019646,0.141648,0.080192,0.064250,0.023301,0.082798,0.125770,...,0.007082,0.031349,0.137345,0.030877,0.095259,https://upload.wikimedia.org/wikipedia/commons...,"Shek Pik Reservoir, with Shek Pik Prison visib...",Shek Pik is an area located along the southwes...,0.121024,-0.009627
5411974,0.064163,0.140261,0.167117,0.088087,0.101651,0.059989,0.258787,0.109619,0.139930,0.124345,...,0.035289,0.014564,0.147341,0.006568,-0.036180,https://upload.wikimedia.org/wikipedia/commons...,"English: University Avenue, University Of Gla...","This is a list of listed buildings in Glasgow,...",0.180178,0.049142
5411975,0.000244,-0.043786,0.052866,-0.048434,-0.027164,-0.027894,-0.043085,0.009353,-0.053874,0.003057,...,0.058317,-0.107188,-0.113409,-0.003040,-0.091142,https://upload.wikimedia.org/wikipedia/commons...,English: Anonymous 16th century Portuguese il...,"The Códice Casanatense, its popular Portuguese...",0.054884,-0.064157
5411976,0.081584,0.030935,0.149223,0.040963,0.050116,0.186897,0.079819,0.104817,0.024297,0.135771,...,0.087509,0.023603,0.009708,0.147209,-0.045105,https://upload.wikimedia.org/wikipedia/commons...,Quagliotto at the 2018 European Road Cycling C...,"Nadia Quagliotto is an Italian racing cyclist,...",0.127953,0.110020


In [ ]:
csim_df.describe()

In [25]:
# df_p = csim_df[csim_df['concept_caption_score'] >= csim_df['concept_caption_score'].mean()]
df_p = csim_df[csim_df['concept_caption_score'] >= 0.134]
df_p = df_p.reset_index(drop=True)
df_p

,196_caption_score,412_caption_score,172_caption_score,363_caption_score,198_caption_score,114_caption_score,419_caption_score,294_caption_score,388_caption_score,314_caption_score,...,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,image_url,caption,context,concept_caption_score,concept_context_score
0,-0.059051,0.034478,0.017459,0.072293,0.042578,-0.037894,0.066543,0.081917,0.112965,-0.000315,...,0.075357,-0.002062,0.038552,0.175929,0.096650,https://upload.wikimedia.org/wikipedia/commons...,English: A headshot of Peter Weinstein,Peter M. Weinstein is an American politician i...,0.151517,0.155621
1,0.113252,0.179848,0.162650,0.118061,0.312920,0.144430,0.225542,0.120647,0.122567,0.213548,...,0.026096,0.060794,0.090376,0.071820,0.106896,https://upload.wikimedia.org/wikipedia/commons...,Exterior of Lone Star Park English: Lone Star ...,Lone Star Park is a horse racing track and ent...,0.224927,0.040659
2,0.129829,0.031143,0.013498,-0.028030,0.086818,0.067837,0.088030,-0.006901,0.007030,0.077067,...,0.038772,0.083501,0.006546,0.013055,-0.022351,https://upload.wikimedia.org/wikipedia/commons...,English: Robert Hooks,"Robert Hooks is an American actor, producer, a...",0.182111,0.036132
3,0.137582,0.177523,0.207521,0.109279,0.183949,0.168617,0.218117,0.136698,0.086362,0.251034,...,0.030982,0.069321,0.251760,0.020972,0.044483,https://upload.wikimedia.org/wikipedia/commons...,English: Side of the Obelisk facing Hyde Park...,The Hyde Park Obelisk is a heritage-listed obe...,0.218246,0.139382
4,0.049927,0.130767,0.158559,0.042664,0.065716,0.142656,0.184495,0.086053,0.024383,0.139034,...,0.097181,0.135434,-0.053385,-0.041917,-0.033439,https://upload.wikimedia.org/wikipedia/commons...,Coppola at the 2001 Cannes Film Festival Coppo...,Francis Ford Coppola is a retired American fil...,0.166897,0.043755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355833,0.088775,0.078180,0.104002,0.070440,0.024088,0.068000,0.119280,0.069382,0.015567,0.134327,...,0.026237,-0.036236,0.057496,-0.021941,0.005434,https://upload.wikimedia.org/wikipedia/commons...,"en:Manchester Town Hall (en:Manchester, en:En...",A list of permanent working clocks with the la...,0.196253,0.078518
1355834,0.147421,0.137543,0.220968,0.192709,0.167448,0.231680,0.181678,0.163100,0.070264,0.176476,...,0.013793,0.051057,-0.111905,0.054401,0.076998,https://upload.wikimedia.org/wikipedia/commons...,Deborah Goldberg conducting fieldwork English:...,Deborah Esther Goldberg is an American ecologi...,0.215896,0.049033
1355835,0.125749,0.205701,0.218751,0.052688,0.198685,0.148288,0.185024,0.143636,0.006773,0.173097,...,0.059396,0.105359,0.296614,0.048320,0.058189,https://upload.wikimedia.org/wikipedia/commons...,A Nice tramway car at Place Massena Français&#...,"The Nice tramway is a 27.5-kilometre, triple-l...",0.194443,0.115030
1355836,0.064163,0.140261,0.167117,0.088087,0.101651,0.059989,0.258787,0.109619,0.139930,0.124345,...,0.035289,0.014564,0.147341,0.006568,-0.036180,https://upload.wikimedia.org/wikipedia/commons...,"English: University Avenue, University Of Gla...","This is a list of listed buildings in Glasgow,...",0.180178,0.049142


In [27]:
df_p.describe()

,196_caption_score,412_caption_score,172_caption_score,363_caption_score,198_caption_score,114_caption_score,419_caption_score,294_caption_score,388_caption_score,314_caption_score,...,192_context_score,20_context_score,296_context_score,144_context_score,12_context_score,224_context_score,250_context_score,376_context_score,concept_caption_score,concept_context_score
count,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,...,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06,1.355838e+06
mean,8.559561e-02,8.686949e-02,1.113165e-01,5.740445e-02,9.280093e-02,1.027839e-01,1.135433e-01,9.396847e-02,6.029807e-02,1.049492e-01,...,1.923718e-02,2.708928e-02,5.571533e-02,2.264773e-02,3.161171e-02,4.298287e-02,3.614905e-02,2.557106e-02,1.764577e-01,6.276443e-02
std,7.012287e-02,5.866437e-02,6.498859e-02,5.323604e-02,7.046865e-02,6.140767e-02,7.860132e-02,5.407793e-02,6.309682e-02,7.308457e-02,...,6.044843e-02,6.729007e-02,8.930775e-02,5.915104e-02,7.877738e-02,9.391404e-02,6.996516e-02,7.838058e-02,3.635343e-02,6.590349e-02
min,-1.492556e-01,-1.255416e-01,-1.148668e-01,-1.664211e-01,-1.644311e-01,-1.255432e-01,-1.382757e-01,-1.218244e-01,-1.932573e-01,-1.752686e-01,...,-2.700528e-01,-2.722588e-01,-2.784553e-01,-2.538928e-01,-2.658590e-01,-3.016184e-01,-2.675191e-01,-2.956381e-01,1.340000e-01,-2.344599e-01
25%,3.577911e-02,4.545975e-02,6.611484e-02,2.048744e-02,4.381821e-02,6.141226e-02,5.866365e-02,5.715708e-02,1.744766e-02,5.435307e-02,...,-2.188648e-02,-1.804999e-02,-5.748722e-03,-1.798432e-02,-2.255250e-02,-2.260899e-02,-1.217066e-02,-2.787367e-02,1.488910e-01,1.922829e-02
50%,7.626972e-02,8.135188e-02,1.036905e-01,5.396095e-02,8.852602e-02,9.911543e-02,1.033881e-01,8.842104e-02,5.989050e-02,9.831265e-02,...,1.854405e-02,2.545261e-02,4.851583e-02,2.138509e-02,2.751547e-02,3.533991e-02,3.457581e-02,2.304182e-02,1.672995e-01,6.164784e-02
75%,1.254801e-01,1.226350e-01,1.480672e-01,9.003481e-02,1.365627e-01,1.395021e-01,1.575852e-01,1.243169e-01,1.022901e-01,1.486356e-01,...,5.941956e-02,6.989236e-02,1.075833e-01,6.125178e-02,8.111304e-02,1.001635e-01,8.310743e-02,7.678358e-02,1.945790e-01,1.050258e-01
max,7.550066e-01,6.103925e-01,6.149387e-01,5.102714e-01,6.104500e-01,7.142168e-01,6.140311e-01,6.116345e-01,5.387621e-01,6.769031e-01,...,5.363311e-01,5.145766e-01,5.608774e-01,4.586499e-01,6.041094e-01,5.891490e-01,4.173216e-01,5.398021e-01,7.286295e-01,6.892114e-01


In [29]:
df_dict = dict()
for col in caption_score_list:
    num, _ = col.split('_', 1)
    c_list = [col] + [f'{num}_context_score'] + im_c + cap_s + con_s
    df = df_p[c_list]
    df = df[df[col] > df[col].quantile(0.9999)]
#     df = df[df[col] > 0.55]
    df = df.sort_values(by=[col], ascending=False)
    df_dict[col] = df

In [31]:
df_dict[col]

,376_caption_score,376_context_score,image_url,caption,context,concept_caption_score,concept_context_score
870052,0.567895,0.407775,https://upload.wikimedia.org/wikipedia/commons...,English: Recreation of Minnie Mouse's signatu...,This is a list of characters that have appeare...,0.189813,0.156888
788926,0.567709,0.407775,https://upload.wikimedia.org/wikipedia/commons...,English: Recreation of Mickey Mouse's signatu...,This is a list of characters that have appeare...,0.180582,0.156888
841923,0.543113,0.423695,https://upload.wikimedia.org/wikipedia/commons...,Cars 623 and 717 passing on the Red Car Troll...,"The Red Car Trolley is a 1,000 mm metre gauge ...",0.200099,0.132485
667765,0.537630,0.350208,https://upload.wikimedia.org/wikipedia/commons...,Disney villains at Disneyland's Mickey's Hallo...,Mickey's Halloween Party was an annual Hallowe...,0.199137,0.043375
1268785,0.533986,0.539802,https://upload.wikimedia.org/wikipedia/commons...,Theatrical release poster English: Poster for ...,Trolley Troubles is a 1927 animated short subj...,0.172679,0.117095
...,...,...,...,...,...,...,...
597662,0.441235,0.146075,https://upload.wikimedia.org/wikipedia/commons...,English: Iconic Toy Train Station,"The Railway Museum at Mysore, India is an outd...",0.241279,0.132612
1305427,0.440948,0.407775,https://upload.wikimedia.org/wikipedia/commons...,English: Recreation of Tweedledee's signature...,This is a list of characters that have appeare...,0.151732,0.156888
1104752,0.440487,0.061867,https://upload.wikimedia.org/wikipedia/commons...,"English: A Disney bus in Walt Disney World, F...",The Low Floor Series bus is a series of transi...,0.176181,0.056294
937790,0.439831,-0.123846,https://upload.wikimedia.org/wikipedia/en/9/93...,"The official school mascot, Willy. He is the o...","ICCT Colleges Foundation Inc., is a tertiary e...",0.211186,-0.042735


In [73]:
from IPython.display import display

In [34]:
for el in df_dict:
    t,_ = el.split('_', 1)
#     print(t)
    df_dict[el].columns = [f'{t}_caption_sim', f'{t}_context_sim', 'image_url',
                          'caption', 'context', f'ped_caption_sim', f'ped_context_sim']
    display(df_dict[el].describe().round(2))
    print('\n')

,196_caption_sim,196_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.56,0.30,0.44,0.23
std,0.05,0.24,0.11,0.19
min,0.51,-0.14,0.24,-0.06
25%,0.51,0.11,0.35,0.08
50%,0.54,0.25,0.43,0.16
75%,0.57,0.46,0.52,0.36
max,0.76,0.75,0.73,0.69


,412_caption_sim,412_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.46,0.19,0.52,0.19
std,0.04,0.16,0.08,0.17
min,0.43,-0.09,0.20,-0.07
25%,0.44,0.06,0.48,0.07
50%,0.45,0.17,0.53,0.13
75%,0.47,0.30,0.56,0.26
max,0.61,0.65,0.73,0.69


,172_caption_sim,172_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.50,0.18,0.50,0.17
std,0.03,0.16,0.09,0.17
min,0.46,-0.07,0.25,-0.08
25%,0.47,0.05,0.44,0.06
50%,0.49,0.14,0.52,0.11
75%,0.51,0.30,0.55,0.25
max,0.61,0.56,0.73,0.69


,363_caption_sim,363_context_sim,ped_caption_sim,ped_context_sim
count,135.00,135.00,135.00,135.00
mean,0.37,0.18,0.37,0.17
std,0.02,0.12,0.14,0.13
min,0.35,-0.06,0.14,-0.06
25%,0.35,0.09,0.28,0.09
50%,0.37,0.18,0.32,0.15
75%,0.38,0.28,0.48,0.21
max,0.51,0.50,0.73,0.68


,198_caption_sim,198_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.49,0.21,0.36,0.14
std,0.03,0.15,0.12,0.10
min,0.45,-0.14,0.15,-0.04
25%,0.46,0.11,0.27,0.08
50%,0.48,0.18,0.33,0.13
75%,0.50,0.31,0.40,0.21
max,0.61,0.61,0.73,0.50


...

,144_caption_sim,144_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.37,0.07,0.32,0.09
std,0.02,0.09,0.14,0.10
min,0.35,-0.12,0.15,-0.23
25%,0.35,0.02,0.22,0.04
50%,0.36,0.06,0.28,0.07
75%,0.38,0.12,0.41,0.13
max,0.44,0.46,0.73,0.54


,12_caption_sim,12_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.47,0.23,0.30,0.08
std,0.04,0.13,0.14,0.09
min,0.43,-0.07,0.14,-0.06
25%,0.44,0.12,0.20,0.02
50%,0.46,0.23,0.26,0.07
75%,0.48,0.33,0.35,0.12
max,0.63,0.49,0.73,0.46


,224_caption_sim,224_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.50,0.32,0.27,0.12
std,0.03,0.12,0.11,0.07
min,0.48,0.02,0.14,-0.02
25%,0.49,0.26,0.19,0.07
50%,0.49,0.34,0.22,0.12
75%,0.51,0.40,0.29,0.16
max,0.64,0.59,0.67,0.37


,250_caption_sim,250_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.34,0.16,0.23,0.07
std,0.02,0.09,0.12,0.09
min,0.31,-0.08,0.13,-0.08
25%,0.32,0.10,0.16,0.02
50%,0.33,0.16,0.19,0.06
75%,0.34,0.21,0.24,0.11
max,0.45,0.37,0.65,0.50


,376_caption_sim,376_context_sim,ped_caption_sim,ped_context_sim
count,136.00,136.00,136.00,136.00
mean,0.47,0.30,0.19,0.10
std,0.03,0.15,0.04,0.07
min,0.44,-0.12,0.14,-0.15
25%,0.45,0.19,0.16,0.05
50%,0.46,0.37,0.18,0.10
75%,0.48,0.41,0.21,0.16
max,0.57,0.54,0.30,0.27


In [42]:
topics = get_topics(concept='pedestrian')

In [43]:
topics

{196: {'cross',
  'crossing',
  'crosswalk',
  'pedestrian',
  'signal',
  'stripe',
  'traffic'},
 412: {'arrest',
  'pedestrian',
  'perp',
  'pers',
  'walk',
  'walkability',
  'walkable'},
 172: {'car',
  'lane',
  'pedestrian',
  'road',
  'street',
  'traffic',
  'transportation',
  'vehicle'},
 363: {'assault',
  'bystander',
  'intervene',
  'intervention',
  'pedestrian',
  'stalk',
  'stalker',
  'victim',
  'witness'},
 198: {'car',
  'garage',
  'park',
  'parking',
  'parkjockey',
  'pedestrian',
  'tow',
  'vehicle'},
 114: {'athlete',
  'disability',
  'paralympic',
  'paralympics',
  'pedestrian',
  'sport',
  'sportspeople',
  'wheelchair'},
 419: {'avenue',
  'boulevard',
  'intersection',
  'manhattan',
  'pedestrian',
  'road',
  'street',
  'suffix'},
 294: {'drive',
  'fine',
  'licence',
  'offence',
  'pedestrian',
  'penalty',
  'reckless',
  'speed',
  'ticket',
  'traffic',
  'violation'},
 388: {'brownian',
  'diffusion',
  'distribution',
  'markov',
  'pe

In [50]:
def simple_search(context, t):
    percentage = 0
    words = list(topics[t])
    for w in words:
        if w in context:
            percentage = percentage + 1
    return float(percentage/len(words))

In [53]:
simple_search(df_dict[df_key]['context'].tolist()[0], 412)

0.2857142857142857

In [54]:
for t in topics:
    df_key = str(t) + '_caption_score'
    df_dict[df_key]['context_gt'] = df_dict[df_key]['context'].apply(simple_search, t=t)


In [56]:
df_dict['412_caption_score'].describe().round(2)

,412_caption_sim,412_context_sim,ped_caption_sim,ped_context_sim,context_gt
count,136.00,136.00,136.00,136.00,136.00
mean,0.46,0.19,0.52,0.19,0.12
std,0.04,0.16,0.08,0.17,0.15
min,0.43,-0.09,0.20,-0.07,0.00
25%,0.44,0.06,0.48,0.07,0.00
50%,0.45,0.17,0.53,0.13,0.07
75%,0.47,0.30,0.56,0.26,0.14
max,0.61,0.65,0.73,0.69,0.57


In [57]:
df_dict['412_caption_score']

,412_caption_sim,412_context_sim,image_url,caption,context,ped_caption_sim,ped_context_sim,context_gt
874893,0.610392,0.199226,https://upload.wikimedia.org/wikipedia/commons...,Pedestrian trail English: Pedestrian trail,The Blue Water River Walk is a nearly one mile...,0.704843,0.105616,0.142857
479525,0.586082,0.008303,https://upload.wikimedia.org/wikipedia/commons...,Pedestrian walk in Ferizaj English: Ferizaj City,"Ferizaj, in southeastern Kosovo, is its third-...",0.595274,0.053994,0.000000
1244500,0.573594,0.481619,https://upload.wikimedia.org/wikipedia/commons...,Prohibition of pedestrians (includes any kind ...,Jaywalking occurs when a pedestrian walks in o...,0.548525,0.483317,0.428571
496374,0.565820,0.190722,https://upload.wikimedia.org/wikipedia/commons...,English: Pedestrian crossing,Road signs used by countries in the Americas a...,0.728629,0.157267,0.000000
812924,0.565820,0.160237,https://upload.wikimedia.org/wikipedia/commons...,English: Pedestrian crossing,Road signs in Malaysia are standardised road s...,0.728629,0.152377,0.000000
...,...,...,...,...,...,...,...,...
335906,0.426847,0.367209,https://upload.wikimedia.org/wikipedia/commons...,Construction of the new Covered Pedestrian Br...,The Riverview Covered Pedestrian Bridge is a w...,0.552095,0.452756,0.142857
1084636,0.426847,0.367209,https://upload.wikimedia.org/wikipedia/commons...,Construction of the new Covered Pedestrian Br...,The Riverview Covered Pedestrian Bridge is a w...,0.552095,0.452756,0.142857
276333,0.426847,0.367209,https://upload.wikimedia.org/wikipedia/commons...,Construction of the new Covered Pedestrian Br...,The Riverview Covered Pedestrian Bridge is a w...,0.552095,0.452756,0.142857
814205,0.426273,-0.059679,https://upload.wikimedia.org/wikipedia/commons...,Pedestrian route through the external rotunda,"The Neue Staatsgalerie in Stuttgart, Germany, ...",0.578300,-0.016740,0.000000


In [69]:
pg_dict = dict()
pg_dict['a'] = [68, 112, 56, 224]
pg_dict['b'] = [309, 344, 47, -1, 53]
pg_dict['c'] = [314, 331, 419, 394]
pg_dict['d'] = [98, 196, 202, 294]
pg_dict['e'] = [172, 137, 11, 134, 150, 216]
pg_dict['f'] = [363, 40, 43]
pg_dict['g'] = [441, 376]
pg_dict['h'] = [144, 416, 327]
pg_dict['i'] = [192, 20, 114]
pg_dict['j'] = [345]
pg_dict['k'] = [8, 265]
pg_dict['l'] = [250, 80, 426]
pg_dict['m'] = [169, 99, 378]
pg_dict['n'] = [432, 388, 412]
pg_dict['o'] = [12, 278, 198, 296] 

In [70]:
k_name = ['image_url', 'caption', 'context', 'concept2caption_sim', 
          'concept2context_sim', 'context_gt']
for el in pg_dict:
    pg_dict[el] = [f'{i}_caption_score' for i in pg_dict[el]]
    pg_dict[el] = [df_dict[i] for i in pg_dict[el]]
    pg_dict[el] = [df_.reset_index(drop=True) for df_ in pg_dict[el]]
    

In [71]:
for el in pg_dict:
    t_df = pd.DataFrame(columns = ['topic2caption_sim', 'topic2context_sim'] + k_name)
    for df_ in pg_dict[el]:
        df_.columns = ['topic2caption_sim', 'topic2context_sim'] + k_name
        t_df = pd.concat([t_df, df_], ignore_index=True)
    pg_dict[el] = t_df

In [72]:
pg_dict[el]

,topic2caption_sim,topic2context_sim,image_url,caption,context,concept2caption_sim,concept2context_sim,context_gt
0,0.628946,0.295938,https://upload.wikimedia.org/wikipedia/commons...,Pedestrian trail English: Pedestrian trail,The Blue Water River Walk is a nearly one mile...,0.704843,0.105616,0.125000
1,0.591550,0.071280,https://upload.wikimedia.org/wikipedia/commons...,Walking Trail,"Bagh-e-Jinnah, formerly known as Lawrence Gard...",0.422199,0.072473,0.000000
2,0.591550,0.071280,https://upload.wikimedia.org/wikipedia/commons...,Walking Trail,"Bagh-e-Jinnah, formerly known as Lawrence Gard...",0.422199,0.072473,0.000000
3,0.591550,0.071280,https://upload.wikimedia.org/wikipedia/commons...,Walking Trail,"Bagh-e-Jinnah, formerly known as Lawrence Gard...",0.422199,0.072473,0.000000
4,0.589132,0.471493,https://upload.wikimedia.org/wikipedia/commons...,Hikers awaiting southbound train English: Look...,The Appalachian Trail station is a commuter ra...,0.300308,0.143705,0.250000
...,...,...,...,...,...,...,...,...
539,0.520411,0.370678,https://upload.wikimedia.org/wikipedia/commons...,J.B. Starkey Wilderness Trail English: J.B. St...,Starkey Wilderness Preserve is a public recrea...,0.223965,0.015963,0.333333
540,0.520411,0.294029,https://upload.wikimedia.org/wikipedia/commons...,J.B. Starkey Wilderness Trail English: J.B. St...,The Veterans Expressway and Suncoast Parkway i...,0.223965,0.118015,0.444444
541,0.519816,0.452994,https://upload.wikimedia.org/wikipedia/commons...,"English: National Park Meadow, Madison Juncti...","Mount Haynes el. 8,218 feet is a prominent pea...",0.138482,0.101505,0.111111
542,0.519740,0.444490,http://upload.wikimedia.org/wikipedia/commons/...,English: The terrain park at Great Bear in Si...,"Great Bear Recreation Park, more commonly refe...",0.181156,0.079883,0.000000


In [74]:
for el in pg_dict:
    print(el)
    display(pg_dict[el].describe().round(2))


a


,topic2caption_sim,topic2context_sim,concept2caption_sim,concept2context_sim,context_gt
count,544.00,544.00,544.00,544.00,544.00
mean,0.51,0.29,0.29,0.12,0.19
std,0.04,0.13,0.12,0.07,0.16
min,0.44,-0.08,0.14,-0.07,0.00
25%,0.48,0.21,0.21,0.08,0.10
50%,0.50,0.31,0.26,0.12,0.12
75%,0.54,0.38,0.32,0.16,0.30
max,0.67,0.59,0.73,0.69,0.78


b


,topic2caption_sim,topic2context_sim,concept2caption_sim,concept2context_sim,context_gt
count,680.00,680.00,680.00,680.00,680.00
mean,0.50,0.30,0.30,0.11,0.17
std,0.06,0.19,0.14,0.10,0.16
min,0.41,-0.10,0.13,-0.11,0.00
25%,0.46,0.14,0.19,0.05,0.00
50%,0.48,0.29,0.24,0.10,0.11
75%,0.54,0.43,0.37,0.17,0.30
max,0.74,0.74,0.73,0.56,0.70


c


...

,topic2caption_sim,topic2context_sim,concept2caption_sim,concept2context_sim,context_gt
count,407.00,407.00,407.00,407.00,407.00
mean,0.48,0.26,0.34,0.13,0.19
std,0.05,0.17,0.15,0.11,0.19
min,0.39,-0.10,0.14,-0.14,0.00
25%,0.43,0.12,0.23,0.07,0.00
50%,0.47,0.27,0.26,0.11,0.12
75%,0.52,0.39,0.46,0.17,0.30
max,0.63,0.62,0.73,0.68,0.80


n


,topic2caption_sim,topic2context_sim,concept2caption_sim,concept2context_sim,context_gt
count,408.00,408.00,408.00,408.00,408.00
mean,0.43,0.19,0.36,0.13,0.08
std,0.05,0.15,0.17,0.13,0.12
min,0.34,-0.14,0.13,-0.14,0.00
25%,0.40,0.07,0.20,0.05,0.00
50%,0.44,0.17,0.33,0.10,0.00
75%,0.46,0.29,0.52,0.17,0.14
max,0.61,0.65,0.73,0.69,0.57


o


,topic2caption_sim,topic2context_sim,concept2caption_sim,concept2context_sim,context_gt
count,544.00,544.00,544.00,544.00,544.00
mean,0.52,0.24,0.31,0.10,0.14
std,0.05,0.15,0.14,0.10,0.17
min,0.43,-0.14,0.14,-0.09,0.00
25%,0.47,0.12,0.20,0.04,0.00
50%,0.52,0.23,0.27,0.09,0.11
75%,0.56,0.37,0.39,0.14,0.25
max,0.71,0.61,0.73,0.57,0.88


In [33]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
df_ = df_dict[el][df_dict[el]['376_score'] >= 0.5]
df_['caption']

4160119    A Peanuts character at the Playhouse Theatre i...
2391380     English: Recreation of Minnie Mouse's signatu...
2065916     English: Recreation of Mickey Mouse's signatu...
4228353    Some popular Disney characters (from left to r...
2278672     Cars 623 and 717 passing on the Red Car Troll...
1581985    Disney villains at Disneyland's Mickey's Hallo...
4525356    Theatrical release poster English: Poster for ...
1207889    Disney with Mickey Mouse English: Walt Disney ...
3253565    The Walt Disney World Railroad The Roy O.Disne...
2202810     English: Recreation of Goofy's signature from...
4204084     English: Recreation of Donald Duck's signatur...
3067569     English: Mickey and Minnie's Runaway Railway ...
699304           The Golden Mickeys at Hong Kong Disneyland 
4173529    Peanuts creator Charles M. Schulz advised Jim ...
3656634    Mickey's Toontown in Disneyland w:Toontown w:D...
1197681    The loading area in Disneyland, California. En...
4448351     English: Rec

In [34]:
pd.reset_option('max_columns')
pd.reset_option('display.max_rows')

In [18]:
csim_des = csim_df.describe().round(2)

In [19]:
pd.set_option('display.max_columns', None)
csim_des


,196_score,412_score,172_score,363_score,198_score,114_score,419_score,294_score,388_score,314_score,98_score,80_score,134_score,432_score,394_score,53_score,56_score,150_score,278_score,-1_score,43_score,265_score,345_score,378_score,11_score,331_score,202_score,169_score,327_score,309_score,68_score,216_score,40_score,99_score,426_score,344_score,47_score,8_score,416_score,112_score,137_score,441_score,192_score,20_score,296_score,144_score,12_score,224_score,250_score,376_score
count,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00,5411978.00
mean,0.03,0.03,0.05,0.01,0.04,0.05,0.05,0.04,0.02,0.05,0.06,0.01,0.07,0.05,0.07,0.03,0.05,0.04,0.09,0.08,0.05,0.03,0.02,0.06,0.05,0.07,0.01,0.04,0.09,0.04,0.04,0.06,0.07,0.03,0.02,0.08,0.08,0.00,0.05,0.03,0.07,0.04,0.02,0.04,0.07,0.05,0.02,0.06,0.03,0.06
std,0.07,0.06,0.07,0.06,0.07,0.07,0.08,0.06,0.07,0.07,0.07,0.07,0.08,0.07,0.08,0.07,0.08,0.07,0.08,0.08,0.07,0.06,0.07,0.09,0.08,0.09,0.06,0.07,0.08,0.08,0.07,0.08,0.07,0.07,0.06,0.08,0.09,0.07,0.06,0.07,0.07,0.08,0.06,0.06,0.09,0.07,0.07,0.08,0.07,0.08
min,-0.25,-0.24,-0.25,-0.25,-0.28,-0.27,-0.24,-0.23,-0.28,-0.25,-0.24,-0.28,-0.28,-0.26,-0.32,-0.25,-0.28,-0.25,-0.27,-0.26,-0.26,-0.25,-0.30,-0.28,-0.26,-0.27,-0.25,-0.27,-0.26,-0.27,-0.28,-0.27,-0.26,-0.26,-0.25,-0.28,-0.27,-0.29,-0.27,-0.26,-0.28,-0.29,-0.27,-0.33,-0.26,-0.25,-0.30,-0.28,-0.24,-0.28
25%,-0.01,-0.02,0.00,-0.03,-0.02,-0.00,0.00,-0.00,-0.03,0.00,0.01,-0.04,0.02,-0.00,0.01,-0.02,-0.00,-0.00,0.03,0.02,0.00,-0.01,-0.02,0.00,-0.01,0.01,-0.03,-0.01,0.04,-0.02,-0.01,0.00,0.02,-0.01,-0.02,0.03,0.02,-0.04,0.01,-0.02,0.02,-0.01,-0.02,-0.01,0.01,0.00,-0.03,-0.00,-0.01,0.01
50%,0.03,0.03,0.04,0.01,0.03,0.04,0.05,0.03,0.02,0.05,0.05,0.01,0.07,0.05,0.06,0.02,0.05,0.04,0.08,0.08,0.05,0.03,0.02,0.05,0.04,0.07,0.01,0.04,0.09,0.03,0.03,0.05,0.06,0.03,0.02,0.08,0.07,0.00,0.05,0.02,0.06,0.04,0.02,0.04,0.07,0.05,0.02,0.05,0.03,0.06
75%,0.07,0.07,0.09,0.05,0.08,0.09,0.10,0.07,0.06,0.10,0.10,0.05,0.12,0.09,0.12,0.07,0.10,0.08,0.14,0.13,0.09,0.07,0.07,0.11,0.09,0.13,0.05,0.09,0.14,0.08,0.08,0.11,0.11,0.07,0.06,0.13,0.13,0.05,0.09,0.07,0.11,0.09,0.06,0.08,0.12,0.09,0.07,0.11,0.08,0.11
max,0.76,0.61,0.61,0.51,0.61,0.71,0.61,0.61,0.55,0.68,0.73,0.51,0.66,0.53,0.68,0.57,0.67,0.64,0.71,0.57,0.64,0.56,0.67,0.63,0.61,0.61,0.62,0.62,0.63,0.60,0.57,0.63,0.61,0.57,0.48,0.58,0.74,0.58,0.66,0.62,0.61,0.57,0.53,0.62,0.67,0.44,0.63,0.64,0.45,0.58


In [20]:
pd.reset_option('max_columns')

In [17]:
col_list = [el for el in df.columns if 'score' in el]
print(col_list)

['196_score', '412_score', '172_score', '363_score', '198_score', '114_score', '419_score', '294_score', '388_score', '314_score', '98_score', '80_score', '134_score', '432_score', '394_score', '53_score', '56_score', '150_score', '278_score', '-1_score', '43_score', '265_score', '345_score', '378_score', '11_score', '331_score', '202_score', '169_score', '327_score', '309_score', '68_score', '216_score', '40_score', '99_score', '426_score', '344_score', '47_score', '8_score', '416_score', '112_score', '137_score', '441_score', '192_score', '20_score', '296_score', '144_score', '12_score', '224_score', '250_score', '376_score']


In [18]:
df.columns

Index(['index', 'language', 'page_url', 'image_url', 'page_title',
       'section_title', 'hierarchical_section_title',
       'caption_reference_description', 'caption_attribution_description',
       'caption_alt_text_description', 'mime_type', 'original_height',
       'original_width', 'is_main_image', 'attribution_passes_lang_id',
       'page_changed_recently', 'context_page_description',
       'context_section_description', 'caption', 'context', '196_score',
       '412_score', '172_score', '363_score', '198_score', '114_score',
       '419_score', '294_score', '388_score', '314_score', '98_score',
       '80_score', '134_score', '432_score', '394_score', '53_score',
       '56_score', '150_score', '278_score', '-1_score', '43_score',
       '265_score', '345_score', '378_score', '11_score', '331_score',
       '202_score', '169_score', '327_score', '309_score', '68_score',
       '216_score', '40_score', '99_score', '426_score', '344_score',
       '47_score', '8_score', '416

In [19]:
df_1 = df[['196_score', 'image_url', 'caption']]
df_1 = df_1.sort_values(by=['196_score'], ascending=False)
df_1 = df_1.reset_index(drop=True)
df_1

,196_score,image_url,caption
0,0.627881,https://upload.wikimedia.org/wikipedia/commons...,"New pedestrian crossing, Barnards Green"
1,0.614582,https://upload.wikimedia.org/wikipedia/commons...,Crosswalk in San Francisco English: Pedestrian...
2,0.566653,https://upload.wikimedia.org/wikipedia/commons...,"A typical ""L-shape"" of tactile paving at a con..."
3,0.561883,https://upload.wikimedia.org/wikipedia/commons...,Pedestrian crossing view of Puente de la Unida...
4,0.560429,https://upload.wikimedia.org/wikipedia/commons...,"""A Pedestrian Was Killed Crossing Here"" sign o..."
...,...,...,...
542588,-0.219343,https://upload.wikimedia.org/wikipedia/commons...,Emperor Hanazono after taking the tonsure. Eng...
542589,-0.221817,https://upload.wikimedia.org/wikipedia/commons...,Arabia has been gradually drying out since the...
542590,-0.223173,https://upload.wikimedia.org/wikipedia/commons...,English: The steamer H.D. Coffinberry before ...
542591,-0.225270,https://upload.wikimedia.org/wikipedia/en/a/a8...,Bucket detail on a small turbine.


In [23]:
import urllib
import base64
import copy
from os.path import exists
from time import sleep
img_root = '/raid/AISSEL/Hamed/datasets/wit/images/'

In [54]:
def get_image(link):
    URL = copy.copy(link)
    link = link.encode("utf-8")
    b64 = base64.b64encode(link)
    s_64 = str(b64)
    s_64 = s_64[2:-1]
    im_path = img_root + s_64 + '.jpg'
    if not exists(im_path):
        try:
            with urllib.request.urlopen(URL) as url:
                with open(im_path, 'wb') as f:
                    f.write(url.read())
            print(f'we download {str(base64.b64decode(s_64))[2:-1]}')
        except OSError as oserr:
            print(f'File name too long')
        except Exception as e:
            print(e)
            print(f'I could not download {str(base64.b64decode(s_64))[2:-1]}')
            sleep(5)
    else:
#         print(f'{im_path}\nalready downloaded please check it')
        pass
        
#     print(f'we decode it back from {str(base64.b64decode(s_64))}')
#     return str(base64.b64decode(s_64))[2:-1]

In [58]:
import requests

session = requests.session()
# session.proxies = {}
# session.proxies['http'] = 'socks5h://localhost:9050'
# session.proxies['https'] = 'socks5h://localhost:9050'
# url = 'https://upload.wikimedia.org/wikipedia/commons/8/8c/Peter_Weinstein_New.jpg'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 '
                         'Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'
           }

In [82]:
def get_image(link):
    URL = copy.copy(link)
    link = link.encode("utf-8")
    b64 = base64.b64encode(link)
    s_64 = str(b64)
    s_64 = s_64[2:-1]
    im_path = img_root + s_64 + '.jpg'
    if not exists(im_path):
        try:
            r = session.get(URL, headers=headers)
            with open(im_path, 'wb') as f:
                f.write(r.content)
            print(f'we download {str(base64.b64decode(s_64))[2:-1]}')
        except OSError as error:
#             print(error)
            pass
        except Exception as e:
            print(e)
            print(f'I could not download {str(base64.b64decode(s_64))[2:-1]}')
            sleep(5)
    else:
        pass
#         print(f'{im_path}\nalready downloaded please check it')


In [27]:
ll = 'https://upload.wikimedia.org/wikipedia/commons/8/8c/Peter_Weinstein_New.jpg'
get_image(ll)


/raid/AISSEL/Hamed/datasets/wit/images/aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcGVkaWEvY29tbW9ucy84LzhjL1BldGVyX1dlaW5zdGVpbl9OZXcuanBn.jpg
already downloaded please check it


In [100]:
df_p[['image_url', 'caption']]

,image_url,caption
0,https://upload.wikimedia.org/wikipedia/commons...,English: A headshot of Peter Weinstein
1,https://upload.wikimedia.org/wikipedia/commons...,Exterior of Lone Star Park English: Lone Star ...
2,https://upload.wikimedia.org/wikipedia/commons...,English: Robert Hooks
3,https://upload.wikimedia.org/wikipedia/commons...,English: Side of the Obelisk facing Hyde Park...
4,https://upload.wikimedia.org/wikipedia/commons...,Coppola at the 2001 Cannes Film Festival Coppo...
...,...,...
1355833,https://upload.wikimedia.org/wikipedia/commons...,"en:Manchester Town Hall (en:Manchester, en:En..."
1355834,https://upload.wikimedia.org/wikipedia/commons...,Deborah Goldberg conducting fieldwork English:...
1355835,https://upload.wikimedia.org/wikipedia/commons...,A Nice tramway car at Place Massena Français&#...
1355836,https://upload.wikimedia.org/wikipedia/commons...,"English: University Avenue, University Of Gla..."


In [103]:
def b64(link):
    link = link.encode("utf-8")
    return str(base64.b64encode(link))[2:-1]

In [106]:
df = pd.DataFrame(columns=['uniq-id'])
uid = df_p["image_url"].apply(b64).values.tolist()
cap = df_p["caption"].values.tolist()

In [108]:
df_o = pd.DataFrame(columns=['uniq_id', 'image_id', 'caption', 'predicted object labels'])
df_o['uniq_id'] = uid
df_o['image_id'] = ""
df_o["caption"] = cap
df_o['predicted object labels'] = ""
df_o

,uniq_id,image_id,caption,predicted object labels
0,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,English: A headshot of Peter Weinstein,
1,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,Exterior of Lone Star Park English: Lone Star ...,
2,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,English: Robert Hooks,
3,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,English: Side of the Obelisk facing Hyde Park...,
4,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,Coppola at the 2001 Cannes Film Festival Coppo...,
...,...,...,...,...
1355833,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,"en:Manchester Town Hall (en:Manchester, en:En...",
1355834,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,Deborah Goldberg conducting fieldwork English:...,
1355835,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,A Nice tramway car at Place Massena Français&#...,
1355836,aHR0cHM6Ly91cGxvYWQud2lraW1lZGlhLm9yZy93aWtpcG...,,"English: University Avenue, University Of Gla...",


In [110]:
df_o.to_csv('ofa.tsv', sep="\t", index=False)

In [111]:
! pwd

/home/barzamini/text_processing


In [29]:
df_p["image_url"].to_csv('example.tsv', sep="\t")

0          https://upload.wikimedia.org/wikipedia/commons...
1          https://upload.wikimedia.org/wikipedia/commons...
2          https://upload.wikimedia.org/wikipedia/commons...
3          https://upload.wikimedia.org/wikipedia/commons...
4          https://upload.wikimedia.org/wikipedia/commons...
                                 ...                        
1355833    https://upload.wikimedia.org/wikipedia/commons...
1355834    https://upload.wikimedia.org/wikipedia/commons...
1355835    https://upload.wikimedia.org/wikipedia/commons...
1355836    https://upload.wikimedia.org/wikipedia/commons...
1355837    https://upload.wikimedia.org/wikipedia/commons...
Name: image_url, Length: 1355838, dtype: object

In [96]:
df_p["image_url"].to_csv(f'{root_path}/selected_url.tsv', sep="\t")

In [94]:
df_1 = df_p.iloc[677919:827919,:]
df_1["image_url"].to_csv(f'{root_path}/selected_url_1.tsv', sep="\t")
df_2 = df_p.iloc[827919:977919,:]
df_2["image_url"].to_csv(f'{root_path}/selected_url_2.tsv', sep="\t")
df_3 = df_p.iloc[977919:1127919,:]
df_3["image_url"].to_csv(f'{root_path}/selected_url_3.tsv', sep="\t")
df_4 = df_p.iloc[1127919:1277919,:]
df_4["image_url"].to_csv(f'{root_path}/selected_url_4.tsv', sep="\t")
df_5 = df_p.iloc[1277919:1355838,:]
df_5["image_url"].to_csv(f'{root_path}/selected_url_5.tsv', sep="\t")

In [83]:
for index, row in tqdm(df_p.iterrows(), total=df_p.shape[0]):
    get_image(row["image_url"])

  0%|          | 0/1355838 [00:00<?, ?it/s]

we download http://upload.wikimedia.org/wikipedia/commons/8/8d/Toby_Roth.jpg
we download http://upload.wikimedia.org/wikipedia/commons/0/0e/Hannibal_and_old_Mark_Twain_Memorial_Bridge.jpg
we download http://upload.wikimedia.org/wikipedia/commons/5/54/LRB_Station_03.jpg
we download http://upload.wikimedia.org/wikipedia/commons/c/c8/FR-43-Costaros3.JPG
we download http://upload.wikimedia.org/wikipedia/commons/0/0b/Baltasar_Lobo_%2C_Madrid_Dec_08.jpg
we download http://upload.wikimedia.org/wikipedia/commons/b/b1/Jaguar_XJ6_--_06-09-2011.jpg
we download http://upload.wikimedia.org/wikipedia/commons/7/7d/Map_of_Ontario_ALGOMA.svg
we download http://upload.wikimedia.org/wikipedia/commons/7/73/Dingle_Peninsula_Coast.jpg
we download http://upload.wikimedia.org/wikipedia/commons/2/22/Jeff-parke-2009-04-08.jpg
we download http://upload.wikimedia.org/wikipedia/commons/0/07/Union_Pacific_Railroad_City_of_Portland_Gold_Room.JPG
we download https://upload.wikimedia.org/wikipedia/commons/8/8d/USA_Obe

we download https://upload.wikimedia.org/wikipedia/commons/b/b1/Building_at_1401-1407_Elmwood_2.JPG
we download https://upload.wikimedia.org/wikipedia/commons/6/69/Mark_Martin_2009.jpg
we download https://upload.wikimedia.org/wikipedia/commons/a/a1/Easby_Hall_%28geograph_4472078%29.jpg
we download https://upload.wikimedia.org/wikipedia/commons/9/99/Akbar%27s_amrour.jpg
we download https://upload.wikimedia.org/wikipedia/commons/a/a1/Hezbollah_ZU-23-2_anti-aircraft_gun.jpg
we download https://upload.wikimedia.org/wikipedia/commons/3/3e/Esgl%C3%A9sia_parroquial_de_Sant_Miquel_Arc%C3%A0ngel_de_Pobla_Tornesa.JPG
we download http://upload.wikimedia.org/wikipedia/commons/c/c4/Hampton_Gray_bust.jpg
we download https://upload.wikimedia.org/wikipedia/commons/5/5e/Buffalo_City_Hall_-_Council_Chambers.jpg
we download https://upload.wikimedia.org/wikipedia/commons/c/c9/SaoPaulo_Municip_SaoRoque.svg
we download https://upload.wikimedia.org/wikipedia/commons/7/71/WilliamThomas.jpg
we download https:/

we download https://upload.wikimedia.org/wikipedia/commons/5/5f/The_entrance_to_the_Circle_Bar_B_Reserve_from_the_Fort_Fraser-Circle_Bar_B_connector_trail.png
we download https://upload.wikimedia.org/wikipedia/commons/a/a3/Glen_Line_Building_Shanghai.jpg
we download http://upload.wikimedia.org/wikipedia/commons/8/84/India_defence_savings_stamp_1943.jpg
we download https://upload.wikimedia.org/wikipedia/commons/4/4f/Flag_of_the_United_States_Department_of_Transportation.svg
we download https://upload.wikimedia.org/wikipedia/commons/0/08/Zhang_Renjun.jpg
we download https://upload.wikimedia.org/wikipedia/commons/4/46/Wade_Lund_Camp_Ripley_summer_2009_219.jpg
we download https://upload.wikimedia.org/wikipedia/commons/f/f2/Motorola_Droid_Mini_XT1030_Red.jpg
we download https://upload.wikimedia.org/wikipedia/commons/1/1b/No%C3%ABl_Olivier_1908.jpg
we download https://upload.wikimedia.org/wikipedia/commons/f/f7/Oberd%C3%B6bling_mit_Pfarrkirche_1830.jpg
we download https://upload.wikimedia.or

we download https://upload.wikimedia.org/wikipedia/commons/d/d8/Trossin_Dahlenberg_Nachbarbierbrunnen.jpg
we download https://upload.wikimedia.org/wikipedia/commons/b/bc/Padlockable_Exchange_Box_.jpg
we download https://upload.wikimedia.org/wikipedia/commons/9/97/Khungi_Pul_%2C_Katlang.jpg
we download https://upload.wikimedia.org/wikipedia/commons/c/c9/Votivskip_i_Grane_kyrkje.jpg
we download https://upload.wikimedia.org/wikipedia/commons/d/d2/Spiral-cone-log.svg
we download https://upload.wikimedia.org/wikipedia/commons/e/e9/Make_sure_to_take_my_best_side%3F_%2832907294474%29.jpg
we download https://upload.wikimedia.org/wikipedia/commons/9/98/JLB-EB_-_20150328_-_Tyler_Cain.jpg
we download http://upload.wikimedia.org/wikipedia/commons/f/fe/BOB_03_17_November_2012.jpg
we download https://upload.wikimedia.org/wikipedia/commons/3/3d/Kalika_Mandir_Dias_View.JPG
we download https://upload.wikimedia.org/wikipedia/commons/7/7d/Donje_Leskovice_-_op%C5%A1tina_Valjevo_-_zapadna_Srbija_-_Panorama

...

we download https://upload.wikimedia.org/wikipedia/commons/9/9e/Ulster_County_New_York_incorporated_and_unincorporated_areas_Saugerties_%28town%29_highlighted.svg
we download https://upload.wikimedia.org/wikipedia/commons/5/52/BlacktownNSWhospital.jpg
we download https://upload.wikimedia.org/wikipedia/commons/0/09/1416Wasserhorst_Kirchwurt.JPG
we download https://upload.wikimedia.org/wikipedia/commons/2/28/Antonio_Juliano_1974.jpg
we download https://upload.wikimedia.org/wikipedia/commons/d/db/Leopaard_CS10_002.jpg
we download https://upload.wikimedia.org/wikipedia/commons/8/81/Walter_Jones_Rock_House.jpg
we download https://upload.wikimedia.org/wikipedia/commons/f/fe/Porsche_Museum_IMG_20141112_123627_%2815851050092%29.jpg
we download https://upload.wikimedia.org/wikipedia/commons/b/b1/Yellow_Lamas_with_Prayer_Wheels.jpg
we download https://upload.wikimedia.org/wikipedia/commons/3/39/Trepimae.JPG
we download https://upload.wikimedia.org/wikipedia/commons/4/46/Plimoth_Plantation_goats.

we download https://upload.wikimedia.org/wikipedia/commons/8/88/Buick_Skylark_Convertible_Coupe_1953.jpg
we download https://upload.wikimedia.org/wikipedia/commons/0/00/Gerald_Albright.jpg
we download https://upload.wikimedia.org/wikipedia/commons/9/98/Reboul-Two_Pigeons.jpg
we download https://upload.wikimedia.org/wikipedia/commons/5/54/Aerial_photo_of_Council_Grove_High_School_Council_Grove_Kansas_9-4-2013.JPG
we download https://upload.wikimedia.org/wikipedia/commons/9/98/142078_passes_Ardwick.jpg
we download https://upload.wikimedia.org/wikipedia/commons/3/38/Asnelles_%2814%29_Fa%C3%A7ade_nord_dela_nef_de_l%27%C3%A9glise.JPG
we download https://upload.wikimedia.org/wikipedia/commons/e/e8/OakFoodAndDrums.JPG
we download https://upload.wikimedia.org/wikipedia/commons/3/3f/Von_Ostensgade_11_%28Drag%C3%B8r%29_01.jpg
we download https://upload.wikimedia.org/wikipedia/commons/4/4e/Marie_Hjelmer-crop.jpg
we download https://upload.wikimedia.org/wikipedia/commons/2/24/Robert_Denning_with_B

we download https://upload.wikimedia.org/wikipedia/commons/a/aa/Anthony_Clarke_at_1996_Atlanta_Paralympics_Games.jpg
we download https://upload.wikimedia.org/wikipedia/commons/8/8d/Altamont_House_September_2019_02.jpg
we download https://upload.wikimedia.org/wikipedia/commons/f/f6/Moshe_Carmel_1955.jpg
we download https://upload.wikimedia.org/wikipedia/commons/d/df/X-26.png
we download https://upload.wikimedia.org/wikipedia/commons/1/1e/Slovenia_-_USA_at_FIFA_World_Cup_2010%2C_out.jpg
we download https://upload.wikimedia.org/wikipedia/commons/b/b8/060912_-_Damien_Bowen_-_3b_-_2012_Summer_Paralympics.jpg
we download https://upload.wikimedia.org/wikipedia/commons/5/5a/LVL3_Artist_of_the_Week-Andrea_Crespo_April_17%2C2019.png
we download https://upload.wikimedia.org/wikipedia/commons/4/49/Supine_and_prone_diagrams-en.svg
we download https://upload.wikimedia.org/wikipedia/commons/0/0a/26%2C_Cariel_Brooks%2C_June_22%2C_2019.jpg
we download https://upload.wikimedia.org/wikipedia/commons/2/29

we download https://upload.wikimedia.org/wikipedia/commons/2/2c/EMU_v_Central_Michigan_051.JPG


KeyboardInterrupt: 